In [1]:
from Dataset_Loader import load_leafs_dataset
from Training import train_cnn
from Model_Evaluation import evaluate_model_with_outputs

from torch import nn, optim, manual_seed, flatten
from Util import apply_conv, apply_pool
import matplotlib.pyplot as plt
from time import time

d:\Downloads\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 20
test_split_size = 0.2

images_train, images_test, labels_train, labels_test = load_leafs_dataset(
    test_split_size,
    batch_size 
)

Reading CSV: 0.253 seconds
Encoding Labels: 0.083 seconds
Resizing Images: 8.161 seconds
Splitting Data: 0.73 seconds
Converting To Tensor: 23.752 seconds
Dividing To Batches: 0.286 seconds


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        width = 1633
        height = 1089
        self.pool1, width, height = apply_pool(width, height, kernel_size=2, stride=2)
        self.pool2, width, height = apply_pool(width, height, kernel_size=2, stride=2)
        self.pool3, width, height = apply_pool(width, height, kernel_size=2, stride=2)
    
        self.conv1, width, height = apply_conv(width, height, 1, 6, kernel_size=3, stride=1, padding=1)
        self.pool4, width, height = apply_pool(width, height, kernel_size=2, stride=2)

        self.conv2, width, height = apply_conv(width, height, 6, 16, kernel_size=3, stride=1, padding=1)
        self.pool5, width, height = apply_pool(width, height, kernel_size=2, stride=2)

        self.fc1 = nn.Linear(16*width*height, 1000)
        self.fc2 = nn.Linear(1000, 99)

    def forward(self, x):
        x = self.pool1(x)
        x = self.pool2(x)
        x = self.pool3(x)

        x = nn.functional.relu(self.conv1(x))
        x = self.pool4(x)

        x = nn.functional.relu(self.conv2(x))
        x = self.pool5(x)

        x = flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.log_softmax(self.fc2(x), dim=1)
        return x

# Optimizer Analysis

In [ ]:
optimizers = [
    optim.SGD, optim.Adagrad, optim.RMSprop, optim.Adam
]
optimizer_names = [
    'SGD', 'Adagrad', 'RMSprop', 'Adam'
]
fig = plt.figure(figsize=(19, 15))

for i in range(len(optimizers)):
    print(f'Analysing {optimizer_names[i]} Optimizer...')
    manual_seed(1)
    model = CNN()
    optimizer = optimizers[i](model.parameters(), 0.001)
    epochs = 10
    initial_train_accuracy = evaluate_model_with_outputs(model, images_train, labels_train, batch_size, 0)
    initial_test_accuracy = evaluate_model_with_outputs(model, images_test, labels_test, batch_size, 1)

    time1 = time()
    trained_model, losses_train, accuracies_train, accuracies_test = train_cnn(
        model=model,
        images_train=images_train,
        labels_train=labels_train,
        images_test=images_test,
        labels_test=labels_test,
        epochs=epochs,
        batch_size=batch_size,
        lossFunction=nn.CrossEntropyLoss(),
        optimizer=optimizer,
        print_loss=True,
        calc_accuracy=True
    )
    time2 = time()
    print(f'Finished in {round(time2 - time1, 3)} seconds')

    accuracies_train.insert(0, initial_train_accuracy)
    accuracies_test.insert(0, initial_test_accuracy)

    rng_x = range(epochs+1)
    rng_y = range(0, 100, 5)

    ax = fig.add_subplot(221 + i)
    ax.set_title(optimizer_names[i])
    plt.tight_layout()
    plt.plot(rng_x, accuracies_train, label='Training')
    plt.plot(rng_x, accuracies_test, label='Testing')
    plt.xlabel('Epoch')
    plt.xticks(rng_x, rng_x)
    plt.ylabel('Accuracy')
    plt.yticks(rng_y, rng_y)
    plt.grid()
    plt.legend()
plt.show()

# L2 Regularization Coefficient Analysis

In [ ]:
reg_coeffs = [
    0, 0.001, 0.01, 0.1, 1
]
fig = plt.figure(figsize=(19, 15))

for i in range(1, len(reg_coeffs)):
    print(f'Analysing L2 Regularization Coefficient {reg_coeffs[i]}...')
    manual_seed(1)
    model = CNN()
    epochs = 10
    initial_train_accuracy = evaluate_model_with_outputs(model, images_train, labels_train, batch_size, 0)
    initial_test_accuracy = evaluate_model_with_outputs(model, images_test, labels_test, batch_size, 1)

    time1 = time()
    trained_model, losses_train, accuracies_train, accuracies_test = train_cnn(
        model=model,
        images_train=images_train,
        labels_train=labels_train,
        images_test=images_test,
        labels_test=labels_test,
        epochs=epochs,
        batch_size=batch_size,
        lossFunction=nn.CrossEntropyLoss(),
        optimizer=optim.Adam(model.parameters(), 0.001, weight_decay=reg_coeffs[i]),
        print_loss=True,
        calc_accuracy=True
    )
    time2 = time()
    print(f'Finished in {round(time2 - time1, 3)} seconds')

    accuracies_train.insert(0, initial_train_accuracy)
    accuracies_test.insert(0, initial_test_accuracy)

    rng_x = range(epochs+1)
    rng_y = range(0, 100, 5)

    ax = fig.add_subplot(231 + i)
    ax.set_title(f'Reg Coeff: {reg_coeffs[i]}')
    plt.tight_layout()
    plt.plot(rng_x, accuracies_train, label='Training')
    plt.plot(rng_x, accuracies_test, label='Testing')
    plt.xlabel('Epoch')
    plt.xticks(rng_x, rng_x)
    plt.ylabel('Accuracy')
    plt.yticks(rng_y, rng_y)
    plt.grid()
    plt.legend()    
plt.show()